In [34]:
# !pip install -r requirements.txt
# !python -m nltk.downloader stopwords
# !python -m nltk.downloader wordnet
# !python -m nltk.downloader omw-1.4

In [35]:
from nltk.corpus import stopwords
import snscrape.modules.telegram as telegram
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from nltk.stem.wordnet import WordNetLemmatizer

In [36]:

scraper = telegram.TelegramChannelScraper('thebull_crypto')
telegram_df = pd.DataFrame()
for i,tel in enumerate(scraper.get_items()):
  if ( i < 100):
    df = pd.DataFrame([{'Date':tel.date,'Content':tel.content}])
    telegram_df = pd.concat([telegram_df,df],axis=0,ignore_index=True)
  else:
    break
telegram_df

,Date,Content
0,2022-07-20 07:38:22+00:00,#Bitcoin ✅
1,2022-07-20 07:34:41+00:00,Buying more $OGN here as well.
2,2022-07-20 07:30:37+00:00,$BAND still can buy!
3,2022-07-19 21:57:02+00:00,$BAND buying here!
4,2022-07-19 21:42:50+00:00,$JASMY just getting started!
...,...,...
95,2022-06-24 21:12:34+00:00,$TRB seems ready to take off 🚀
96,2022-06-24 21:04:23+00:00,$SOL is moving nicely. Already +36% from entry!
97,2022-06-24 18:21:39+00:00,With only $30m mcap $TRB will go up +100% easily.
98,2022-06-24 18:10:55+00:00,Bitcoin realized 30d volatility hits 12-month ...


In [37]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
# current_date =date.today()
def text_cleaning(text):
    if(text!=None):
        stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
        return normalized.split()

# telegram_df["Cleaned Text"]= telegram_df["Content"]
telegram_df["Cleaned Text"]= telegram_df["Content"].apply(text_cleaning)


In [38]:
telegram_df.iloc[1,:]

Date                 2022-07-20 07:34:41+00:00
Content         Buying more $OGN here as well.
Cleaned Text               [buying, ogn, well]
Name: 1, dtype: object

In [49]:
df = pd.read_csv("Prediction - Prediction.csv")
X = telegram_df.loc[:,"Cleaned Text"]
Y = df.iloc[:,3]
# Y = Y.apply(lambda x : str(x))
Y

0                                 $CLV getting ready 👌🏻
1                        $OGN is on the way to $0,3  👀🚀
2                                        $FIL is a buy.
3                                      Buying $QI here!
4                                     $CHR +13% so far!
                            ...                        
95                   Buying some $TRB on this breakout!
96                                 $OP +20% from entry!
97                                  $ICP is going to $7
98    The #Bitcoin drawdown from ATH has reached 73....
99            $JASMY breakout and retest successfully 🚀
Name: Content, Length: 100, dtype: object

In [42]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=.2)
model = make_pipeline(TfidfVectorizer(lowercase=False), MultinomialNB())
# model.fit(X_train, Y_train)
# pred = model.predict(X_test)
# model.score(pred,Y_test)
X_train

43                                     [keep, eye, dot]
83                                             [qi, 33]
6                                    [fil, time, go, 👀]
3                                  [band, buying, here]
81    [jasmy, im, still, bullish, it, think, gonna, ...
                            ...                        
79                                     [op, still, buy]
25                            [jasmy, double, one, day]
2                                    [band, still, buy]
40                                           [inj, buy]
34    [get, alan, walker, nfts, real, value, coritew...
Name: Cleaned Text, Length: 80, dtype: object

In [ ]:
def my_predictions(my_sentence, model):
    prediction = model.predict([my_sentence])
    return prediction

result=list()
# telegram_df["Cleaned Text"].apply(lambda x: x.lower())
for i in telegram_df["Cleaned Text"]:
    result.append(my_predictions(str(i),model))
telegram_df["Result"]=result
telegram_df

In [ ]:
telegram_df.to_csv("SD.csv")

In [ ]:
# !pip freeze > requirements.txt